<a href="https://colab.research.google.com/github/hernanGONVE/sales-predictions/blob/main/Proyecto1_parte2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot  as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error, mean_absolute_error


In [3]:
df  = pd.read_csv("/content/drive/MyDrive/Data Analytics - Coding Dojo/1. Avance del curso/4. Semana 4: Estatura promedio/1. semana 4: Proyecto 1 - Parte 2/sales_predictions.csv")

In [4]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [5]:
df["productos"] = round(df["Item_Outlet_Sales"] / df["Item_MRP"],0)

Revision de valores faltantes


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
 12  productos                  8523 non-null   float64
dtypes: float64(5), int64(1), object(7)
memory usage:

In [7]:
df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
productos                       0
dtype: int64

Se observa en la columana "Item_Weight" (Peso del producto) y en "Outlet_Size" (Tamanio de la tienda) que existe valores faltantes

In [8]:
df["Item_Identifier"].unique().shape    #Columna 0

(1559,)

Existe un total de 1559 productos

**Columna 1**, presenta valores faltantes los cuales seran completados con los otros valores que tienen el mismo codigo de indentrifiacion, en caso de que exista algun producto que no presente un precio ya existente y solo figure NaN, este se reemplazara por el promedio o la mediana del tipo de **producto**

---



In [9]:
valores_unicos1 =df["Item_Identifier"].unique()
valores_unicos2 =df[~df["Item_Weight"].isna()]["Item_Identifier"].unique()
print(valores_unicos1.shape)
print(valores_unicos2.shape)

(1559,)
(1555,)


- el primer juego de valores es la cantidad de productos que existen
- el segundo juego de valores es la cantidad de productos que existen y sin tienen precio

In [10]:
productos_sin_costo = np.setdiff1d(valores_unicos1,valores_unicos2)
productos_sin_costo

array(['FDE52', 'FDK57', 'FDN52', 'FDQ60'], dtype=object)

En este array se muestra los productos que no tienen precio y no tienen otra fila con el mismo producto que tenga precio para completarlo

In [11]:
df_unicos_NaN = df[df["Item_Identifier"].isin(list(productos_sin_costo))]
df_unicos_NaN

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,productos
927,FDN52,NaN,Regular,0.130933,Frozen Foods,86.9198,OUT027,1985,Medium,Tier 3,Supermarket Type3,1569.9564,18.0
1922,FDK57,NaN,Low Fat,0.079904,Snack Foods,120.0440,OUT027,1985,Medium,Tier 3,Supermarket Type3,4434.2280,37.0
4187,FDE52,NaN,Regular,0.029742,Dairy,88.9514,OUT027,1985,Medium,Tier 3,Supermarket Type3,3453.5046,39.0
5022,FDQ60,NaN,Regular,0.191501,Baking Goods,121.2098,OUT019,1985,Small,Tier 1,Grocery Store,120.5098,1.0


In [12]:
df.groupby("Item_Type")["Item_Weight"].agg(["mean","median"])

,mean,median
Item_Type,,
Baking Goods,12.277108,11.650
Breads,11.346936,10.600
Breakfast,12.768202,10.695
Canned,12.305705,12.150
Dairy,13.426069,13.350
Frozen Foods,12.867061,12.850
Fruits and Vegetables,13.224769,13.100
Hard Drinks,11.400328,10.100
Health and Hygiene,13.142314,12.150


Se observa que la media con la mediana d los productos son similares, asi que para esos 4 productos unicos, utilizaremos el valor promedio del peso de los productos para reemplazarlos

In [13]:
df.loc[df["Item_Identifier"]=="FDN52","Item_Weight"] = df[df["Item_Type"]=="Frozen Foods"]["Item_Weight"].mean()
df.loc[df["Item_Identifier"]=="FDK57","Item_Weight"] = df[df["Item_Type"]=="Snack Foods"]["Item_Weight"].mean()
df.loc[df["Item_Identifier"]=="FDE52","Item_Weight"] = df[df["Item_Type"]=="Dairy"]["Item_Weight"].mean()
df.loc[df["Item_Identifier"]=="FDQ60","Item_Weight"] = df[df["Item_Type"]=="Baking Goods"]["Item_Weight"].mean()

Los otros productos que tienen valores NaN seran reemplazados por productos que tienen el mismo codigo pero que si presentan valores de peso

In [14]:
df.sort_values(["Item_Identifier","Item_Weight"],ascending=[True,False],inplace=True)
df.head(20)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,productos
118,DRA12,11.60,Low Fat,0.041178,Soft Drinks,140.3154,OUT017,2007,NaN,Tier 2,Supermarket Type1,2552.6772,18.0
1197,DRA12,11.60,Low Fat,0.000000,Soft Drinks,141.6154,OUT045,2002,NaN,Tier 2,Supermarket Type1,3829.0158,27.0
1245,DRA12,11.60,Low Fat,0.040912,Soft Drinks,142.3154,OUT013,1987,High,Tier 3,Supermarket Type1,2552.6772,18.0
1693,DRA12,11.60,LF,0.000000,Soft Drinks,141.9154,OUT035,2004,Small,Tier 2,Supermarket Type1,992.7078,7.0
7467,DRA12,11.60,Low Fat,0.041113,Soft Drinks,142.0154,OUT018,2009,Medium,Tier 3,Supermarket Type2,850.8924,6.0
8043,DRA12,11.60,Low Fat,0.068535,Soft Drinks,143.0154,OUT010,1998,NaN,Tier 3,Grocery Store,283.6308,2.0
1148,DRA24,19.35,Regular,0.040154,Soft Drinks,164.6868,OUT017,2007,NaN,Tier 2,Supermarket Type1,1146.5076,7.0
4130,DRA24,19.35,Regular,0.066832,Soft Drinks,163.8868,OUT010,1998,NaN,Tier 3,Grocery Store,327.5736,2.0
4900,DRA24,19.35,Regular,0.039921,Soft Drinks,163.3868,OUT035,2004,Small,Tier 2,Supermarket Type1,3439.5228,21.0
6863,DRA24,19.35,Regular,0.039990,Soft Drinks,165.0868,OUT049,1999,Medium,Tier 1,Supermarket Type1,982.7208,6.0


In [15]:
df["Item_Weight"].fillna(method="ffill", inplace=True)
df.head(20)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,productos
118,DRA12,11.60,Low Fat,0.041178,Soft Drinks,140.3154,OUT017,2007,NaN,Tier 2,Supermarket Type1,2552.6772,18.0
1197,DRA12,11.60,Low Fat,0.000000,Soft Drinks,141.6154,OUT045,2002,NaN,Tier 2,Supermarket Type1,3829.0158,27.0
1245,DRA12,11.60,Low Fat,0.040912,Soft Drinks,142.3154,OUT013,1987,High,Tier 3,Supermarket Type1,2552.6772,18.0
1693,DRA12,11.60,LF,0.000000,Soft Drinks,141.9154,OUT035,2004,Small,Tier 2,Supermarket Type1,992.7078,7.0
7467,DRA12,11.60,Low Fat,0.041113,Soft Drinks,142.0154,OUT018,2009,Medium,Tier 3,Supermarket Type2,850.8924,6.0
8043,DRA12,11.60,Low Fat,0.068535,Soft Drinks,143.0154,OUT010,1998,NaN,Tier 3,Grocery Store,283.6308,2.0
1148,DRA24,19.35,Regular,0.040154,Soft Drinks,164.6868,OUT017,2007,NaN,Tier 2,Supermarket Type1,1146.5076,7.0
4130,DRA24,19.35,Regular,0.066832,Soft Drinks,163.8868,OUT010,1998,NaN,Tier 3,Grocery Store,327.5736,2.0
4900,DRA24,19.35,Regular,0.039921,Soft Drinks,163.3868,OUT035,2004,Small,Tier 2,Supermarket Type1,3439.5228,21.0
6863,DRA24,19.35,Regular,0.039990,Soft Drinks,165.0868,OUT049,1999,Medium,Tier 1,Supermarket Type1,982.7208,6.0


Aqui se termina el completado de valores faltantes en la **Columna 1**

In [16]:
df["Item_Fat_Content"].value_counts()   #Columna 2

Item_Fat_Content
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: count, dtype: int64

In [17]:
df.loc[:,"Item_Fat_Content"].replace("low fat","Low Fat", inplace=True)
df.loc[:,"Item_Fat_Content"].replace("LF","Low Fat", inplace=True)
df.loc[:,"Item_Fat_Content"].replace("reg","Regular", inplace=True)
df["Item_Fat_Content"].value_counts()

Item_Fat_Content
Low Fat    5517
Regular    3006
Name: count, dtype: int64

In [18]:
(df["Item_Visibility"]==0).sum()   #Columna 3

526

El rango de visualizacion de los productos se encuentra entre 0 hasta 0.3228, en donde se encuentra 526 valores igual a cero

In [19]:
df["Item_Type"].value_counts()  #Columna 4

Item_Type
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: count, dtype: int64

In [20]:
df["Item_MRP"].sort_index()  #Columna 5

0       249.8092
1        48.2692
2       141.6180
3       182.0950
4        53.8614
          ...   
8518    214.5218
8519    108.1570
8520     85.1224
8521    103.1332
8522     75.4670
Name: Item_MRP, Length: 8523, dtype: float64

In [21]:
df["Outlet_Identifier"].unique()    #Columna 6

array(['OUT017', 'OUT045', 'OUT013', 'OUT035', 'OUT018', 'OUT010',
       'OUT049', 'OUT019', 'OUT027', 'OUT046'], dtype=object)

Existen en total 10 tiendas

In [22]:
df.columns

Index(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type', 'Item_Outlet_Sales', 'productos'],
      dtype='object')

In [23]:
df["Outlet_Establishment_Year"].value_counts() #Columna 7

Outlet_Establishment_Year
1985    1463
1987     932
2004     930
1999     930
1997     930
2002     929
2009     928
2007     926
1998     555
Name: count, dtype: int64

In [24]:
df["Outlet_Size"].value_counts() #Columna 8

Outlet_Size
Medium    2793
Small     2388
High       932
Name: count, dtype: int64

In [25]:
df[df["Outlet_Size"].isna()]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,productos
118,DRA12,11.60,Low Fat,0.041178,Soft Drinks,140.3154,OUT017,2007,NaN,Tier 2,Supermarket Type1,2552.6772,18.0
1197,DRA12,11.60,Low Fat,0.000000,Soft Drinks,141.6154,OUT045,2002,NaN,Tier 2,Supermarket Type1,3829.0158,27.0
8043,DRA12,11.60,Low Fat,0.068535,Soft Drinks,143.0154,OUT010,1998,NaN,Tier 3,Grocery Store,283.6308,2.0
1148,DRA24,19.35,Regular,0.040154,Soft Drinks,164.6868,OUT017,2007,NaN,Tier 2,Supermarket Type1,1146.5076,7.0
4130,DRA24,19.35,Regular,0.066832,Soft Drinks,163.8868,OUT010,1998,NaN,Tier 3,Grocery Store,327.5736,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
411,NCZ42,10.50,Low Fat,0.000000,Household,238.3248,OUT010,1998,NaN,Tier 3,Grocery Store,711.0744,3.0
1115,NCZ42,10.50,Low Fat,0.011352,Household,236.5248,OUT017,2007,NaN,Tier 2,Supermarket Type1,4029.4216,17.0
4924,NCZ53,9.60,Low Fat,0.040970,Health and Hygiene,188.7214,OUT010,1998,NaN,Tier 3,Grocery Store,376.8428,2.0
214,NCZ54,14.65,Low Fat,0.000000,Household,161.5552,OUT010,1998,NaN,Tier 3,Grocery Store,324.9104,2.0


In [26]:
df["Outlet_Location_Type"].value_counts() #Columna 9

Outlet_Location_Type
Tier 3    3350
Tier 2    2785
Tier 1    2388
Name: count, dtype: int64

In [27]:
df.sort_values(["Outlet_Identifier","Outlet_Size"],ascending=[False,True], inplace=True)

In [28]:
df.drop_duplicates(["Outlet_Identifier","Outlet_Size"])

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,productos
6863,DRA24,19.35,Regular,0.039990,Soft Drinks,165.0868,OUT049,1999,Medium,Tier 1,Supermarket Type1,982.7208,6.0
1876,DRA59,8.27,Regular,0.127928,Soft Drinks,184.8924,OUT046,1997,Small,Tier 1,Supermarket Type1,4442.2176,24.0
1197,DRA12,11.60,Low Fat,0.000000,Soft Drinks,141.6154,OUT045,2002,NaN,Tier 2,Supermarket Type1,3829.0158,27.0
1693,DRA12,11.60,Low Fat,0.000000,Soft Drinks,141.9154,OUT035,2004,Small,Tier 2,Supermarket Type1,992.7078,7.0
4416,DRA24,19.35,Regular,0.039735,Soft Drinks,165.7868,OUT027,1985,Medium,Tier 3,Supermarket Type3,4913.6040,30.0
2879,DRA24,19.35,Regular,0.069909,Soft Drinks,163.2868,OUT019,1985,Small,Tier 1,Grocery Store,491.3604,3.0
7467,DRA12,11.60,Low Fat,0.041113,Soft Drinks,142.0154,OUT018,2009,Medium,Tier 3,Supermarket Type2,850.8924,6.0
118,DRA12,11.60,Low Fat,0.041178,Soft Drinks,140.3154,OUT017,2007,NaN,Tier 2,Supermarket Type1,2552.6772,18.0
1245,DRA12,11.60,Low Fat,0.040912,Soft Drinks,142.3154,OUT013,1987,High,Tier 3,Supermarket Type1,2552.6772,18.0
8043,DRA12,11.60,Low Fat,0.068535,Soft Drinks,143.0154,OUT010,1998,NaN,Tier 3,Grocery Store,283.6308,2.0


Como no se puede recuperar el tipo de dato "Outlet_Size" ya que no se puede encontrar alguna relacion entre esta columna y "Outlet_Location_Type", 	"Outlet_Type" y 	"Item_Outlet_Sales", se procede a eliminar esta columna

In [29]:
df.drop(columns=["Outlet_Size"], inplace=True)
df

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,productos
6863,DRA24,19.350,Regular,0.039990,Soft Drinks,165.0868,OUT049,1999,Tier 1,Supermarket Type1,982.7208,6.0
3927,DRA59,8.270,Regular,0.128127,Soft Drinks,183.6924,OUT049,1999,Tier 1,Supermarket Type1,1295.6468,7.0
3465,DRB01,7.390,Low Fat,0.082367,Soft Drinks,187.7530,OUT049,1999,Tier 1,Supermarket Type1,1518.0240,8.0
6626,DRB13,6.115,Regular,0.007055,Soft Drinks,188.6530,OUT049,1999,Tier 1,Supermarket Type1,3605.3070,19.0
2657,DRB24,8.785,Low Fat,0.020609,Soft Drinks,155.1656,OUT049,1999,Tier 1,Supermarket Type1,4016.1056,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...
8054,NCZ29,15.000,Low Fat,0.119461,Health and Hygiene,126.3362,OUT010,1998,Tier 3,Grocery Store,503.3448,4.0
1224,NCZ30,6.590,Low Fat,0.043828,Household,121.4098,OUT010,1998,Tier 3,Grocery Store,241.0196,2.0
411,NCZ42,10.500,Low Fat,0.000000,Household,238.3248,OUT010,1998,Tier 3,Grocery Store,711.0744,3.0
4924,NCZ53,9.600,Low Fat,0.040970,Health and Hygiene,188.7214,OUT010,1998,Tier 3,Grocery Store,376.8428,2.0


In [30]:
df["Item_Outlet_Sales"] .sort_values() #Columna 11

906        33.2900
6950       33.2900
2571       33.9558
3053       34.6216
7388       35.2874
           ...    
6541    10306.5840
4349    10993.6896
1450    11445.1020
5223    12117.5600
7188    13086.9648
Name: Item_Outlet_Sales, Length: 8523, dtype: float64

#Analisis de los datos

In [31]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,productos
6863,DRA24,19.350,Regular,0.039990,Soft Drinks,165.0868,OUT049,1999,Tier 1,Supermarket Type1,982.7208,6.0
3927,DRA59,8.270,Regular,0.128127,Soft Drinks,183.6924,OUT049,1999,Tier 1,Supermarket Type1,1295.6468,7.0
3465,DRB01,7.390,Low Fat,0.082367,Soft Drinks,187.7530,OUT049,1999,Tier 1,Supermarket Type1,1518.0240,8.0
6626,DRB13,6.115,Regular,0.007055,Soft Drinks,188.6530,OUT049,1999,Tier 1,Supermarket Type1,3605.3070,19.0
2657,DRB24,8.785,Low Fat,0.020609,Soft Drinks,155.1656,OUT049,1999,Tier 1,Supermarket Type1,4016.1056,26.0


In [32]:
df.describe(include='all')

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,productos
count,8523,8523.000000,8523,8523.000000,8523,8523.000000,8523,8523.000000,8523,8523,8523.000000,8523.000000
unique,1559,NaN,2,NaN,16,NaN,10,NaN,3,4,NaN,NaN
top,FDW13,NaN,Low Fat,NaN,Fruits and Vegetables,NaN,OUT027,NaN,Tier 3,Supermarket Type1,NaN,NaN
freq,10,NaN,5517,NaN,1232,NaN,935,NaN,3350,5577,NaN,NaN
mean,NaN,12.875427,NaN,0.066132,NaN,140.992782,NaN,1997.831867,NaN,NaN,2181.288914,15.424850
std,NaN,4.645016,NaN,0.051598,NaN,62.275067,NaN,8.371760,NaN,NaN,1706.499616,9.190713
min,NaN,4.555000,NaN,0.000000,NaN,31.290000,NaN,1985.000000,NaN,NaN,33.290000,1.000000
25%,NaN,8.785000,NaN,0.026989,NaN,93.826500,NaN,1987.000000,NaN,NaN,834.247400,9.000000
50%,NaN,12.650000,NaN,0.053931,NaN,143.012800,NaN,1999.000000,NaN,NaN,1794.331000,15.000000
75%,NaN,16.850000,NaN,0.094585,NaN,185.643700,NaN,2004.000000,NaN,NaN,3101.296400,21.000000


In [33]:
df["Outlet_Type"].value_counts()

Outlet_Type
Supermarket Type1    5577
Grocery Store        1083
Supermarket Type3     935
Supermarket Type2     928
Name: count, dtype: int64

Se realizara un estudio de ventas totales de producto con un subindice de del tipo de producto

# **EDA**

In [34]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,productos
6863,DRA24,19.350,Regular,0.039990,Soft Drinks,165.0868,OUT049,1999,Tier 1,Supermarket Type1,982.7208,6.0
3927,DRA59,8.270,Regular,0.128127,Soft Drinks,183.6924,OUT049,1999,Tier 1,Supermarket Type1,1295.6468,7.0
3465,DRB01,7.390,Low Fat,0.082367,Soft Drinks,187.7530,OUT049,1999,Tier 1,Supermarket Type1,1518.0240,8.0
6626,DRB13,6.115,Regular,0.007055,Soft Drinks,188.6530,OUT049,1999,Tier 1,Supermarket Type1,3605.3070,19.0
2657,DRB24,8.785,Low Fat,0.020609,Soft Drinks,155.1656,OUT049,1999,Tier 1,Supermarket Type1,4016.1056,26.0


In [35]:
import plotly.express as px

fig = px.scatter(df, y="Item_Outlet_Sales", x="productos", color="Outlet_Type")
fig.update_traces(marker_size=7)
fig.update_layout(scattermode="group")
fig.show()

In [388]:
df.groupby(["Outlet_Identifier","Outlet_Establishment_Year"])[["Item_Outlet_Sales"]].agg("sum").sort_values(by  ="Item_Outlet_Sales", ascending=False)

,,Item_Outlet_Sales
Outlet_Identifier,Outlet_Establishment_Year,
OUT027,1985,3.453926e+06
OUT035,2004,2.268123e+06
OUT049,1999,2.183970e+06
OUT017,2007,2.167465e+06
OUT013,1987,2.142664e+06
OUT046,1997,2.118395e+06
OUT045,2002,2.036725e+06
OUT018,2009,1.851823e+06
OUT010,1998,1.883402e+05


Las dos ultimos tiendas son almacenes, asi que por ello se buscara el estudio des tas con otro enfoque

In [389]:
df_high= df[~df["Outlet_Identifier"].isin(["OUT010","OUT019"])]
df_lower= df[df["Outlet_Identifier"].isin(["OUT010","OUT019"])]

In [390]:
import plotly.express as px

da = px.data.medals_long()

fig = px.scatter(df_high, y="Item_Outlet_Sales", x="Item_MRP", color="Outlet_Type")
fig.update_traces(marker_size=7)
fig.update_layout(scattermode="group")
fig.show()

Se puede observar que el mercado de mayores productores se divide en 4 partes principalmente, en donde los productos con un menor costo minorista esta por debajo de los 4000($USD) en ventas de ventas en tiendas particulares, lo mismo sucede en los otros tres grupos, que no superan el costo de 8000, 10000 y el ultimo supera los 12000

In [391]:
import plotly.express as px

da = px.data.medals_long()

fig = px.scatter(df_lower, y="Item_Outlet_Sales", x="Item_MRP", color="Outlet_Type")
fig.update_traces(marker_size=7)
fig.update_layout(scattermode="group")
fig.show()

En cierta manera, los productos de los supermercados tienen la misma tendecia que los productos vendidos en los almacenes

In [392]:
df_high.head()


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,productos
6863,DRA24,19.350,Regular,0.039990,Soft Drinks,165.0868,OUT049,1999,Tier 1,Supermarket Type1,982.7208,6.0
3927,DRA59,8.270,Regular,0.128127,Soft Drinks,183.6924,OUT049,1999,Tier 1,Supermarket Type1,1295.6468,7.0
3465,DRB01,7.390,Low Fat,0.082367,Soft Drinks,187.7530,OUT049,1999,Tier 1,Supermarket Type1,1518.0240,8.0
6626,DRB13,6.115,Regular,0.007055,Soft Drinks,188.6530,OUT049,1999,Tier 1,Supermarket Type1,3605.3070,19.0
2657,DRB24,8.785,Low Fat,0.020609,Soft Drinks,155.1656,OUT049,1999,Tier 1,Supermarket Type1,4016.1056,26.0


In [393]:
df_high.describe(include='all')

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,productos
count,7440,7440.000000,7440,7440.000000,7440,7440.000000,7440,7440.000000,7440,7440,7440.000000,7440.000000
unique,1558,NaN,2,NaN,16,NaN,8,NaN,3,3,NaN,NaN
top,NCJ30,NaN,Low Fat,NaN,Fruits and Vegetables,NaN,OUT027,NaN,Tier 3,Supermarket Type1,NaN,NaN
freq,8,NaN,4812,NaN,1080,NaN,935,NaN,2795,5577,NaN,NaN
mean,NaN,12.863231,NaN,0.060494,NaN,141.094400,NaN,1998.729973,NaN,NaN,2449.340208,17.318683
std,NaN,4.650245,NaN,0.044468,NaN,62.355432,NaN,8.233719,NaN,NaN,1661.527021,8.261646
min,NaN,4.555000,NaN,0.000000,NaN,31.290000,NaN,1985.000000,NaN,NaN,69.243200,2.000000
25%,NaN,8.770000,NaN,0.026037,NaN,93.902150,NaN,1987.000000,NaN,NaN,1193.113600,11.000000
50%,NaN,12.600000,NaN,0.049513,NaN,142.748300,NaN,1999.000000,NaN,NaN,2075.964400,16.000000
75%,NaN,16.850000,NaN,0.087943,NaN,185.864800,NaN,2004.000000,NaN,NaN,3327.668400,22.000000


In [394]:
list_moda=list(df_high["Item_Identifier"].mode())
list_moda

['DRE49',
 'DRN47',
 'FDA04',
 'FDF05',
 'FDG09',
 'FDG24',
 'FDG33',
 'FDL10',
 'FDL34',
 'FDO19',
 'FDO32',
 'FDO52',
 'FDT07',
 'FDT21',
 'FDT49',
 'FDU19',
 'FDW13',
 'FDW49',
 'FDX04',
 'FDX20',
 'NCJ30',
 'NCL31',
 'NCQ06',
 'NCY18']

In [395]:
df_h_moda = df_high[df_high["Item_Identifier"].isin(list_moda)]
df_h_moda.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales,productos
count,192.000000,192.000000,192.000000,192.000000,192.000000,192.000000
mean,11.379792,0.058046,159.782762,1998.750000,2753.502593,17.588542
std,5.220743,0.041237,69.768693,8.248751,1666.312558,7.860863
min,5.365000,0.000000,45.406000,1985.000000,259.662000,2.000000
25%,7.345000,0.026861,103.772050,1994.500000,1287.657200,12.000000
50%,8.635000,0.042556,171.227700,2000.500000,2558.336500,17.000000
75%,16.375000,0.080845,197.484700,2004.750000,3975.824700,22.000000
max,20.750000,0.152262,264.891000,2009.000000,7741.922400,50.000000


In [396]:
df_h_moda.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,productos
4953,DRE49,20.75,Low Fat,0.021283,Soft Drinks,153.5024,OUT049,1999,Tier 1,Supermarket Type1,2428.8384,16.0
1762,DRN47,12.10,Low Fat,0.016853,Hard Drinks,180.6660,OUT049,1999,Tier 1,Supermarket Type1,2876.2560,16.0
5572,FDA04,11.30,Regular,0.066834,Frozen Foods,257.2962,OUT049,1999,Tier 1,Supermarket Type1,2330.9658,9.0
3429,FDF05,17.50,Low Fat,0.026913,Frozen Foods,261.2910,OUT049,1999,Tier 1,Supermarket Type1,3418.8830,13.0
5068,FDG09,20.60,Regular,0.048011,Fruits and Vegetables,187.7556,OUT049,1999,Tier 1,Supermarket Type1,3004.0896,16.0


In [397]:
pie_grafic=df_h_moda.groupby(["Item_Fat_Content","Item_Type","Outlet_Identifier"])["Item_Outlet_Sales"].sum().reset_index()

pie_grafic

,Item_Fat_Content,Item_Type,Outlet_Identifier,Item_Outlet_Sales
0,Low Fat,Baking Goods,OUT013,1081.9250
1,Low Fat,Baking Goods,OUT017,2579.9750
2,Low Fat,Baking Goods,OUT018,1747.7250
3,Low Fat,Baking Goods,OUT027,1331.6000
4,Low Fat,Baking Goods,OUT035,1331.6000
...,...,...,...,...
91,Regular,Seafood,OUT027,3435.5280
92,Regular,Seafood,OUT035,4637.9628
93,Regular,Seafood,OUT045,2233.0932
94,Regular,Seafood,OUT046,3263.7516


In [398]:
import plotly.express as px

fig = px.sunburst(pie_grafic, path=['Item_Type','Item_Fat_Content','Outlet_Identifier'], color='Item_Type',values='Item_Outlet_Sales',title="Productos disponible en todas las tiendas, ingresos por categoria, tienda y producto")
fig.update_layout(width=900, height=900)
fig.show()

Se puede observar que para los productos que se encuentran en todas las tiendas, en las frutas y vegetales, la tienda que tiene la mayor cantidad de ventas es la tienda **OUT027** y que esta categoria es donde se concentra la mayor cantidad de ventas del mercado, luego en las comidas congeladas **OUT018** tiene la mayor cantidad de ingresos.

In [399]:
df_h_no_moda = df_high[~df_high["Item_Identifier"].isin(list_moda)]
df_h_no_moda.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales,productos
count,7248.000000,7248.000000,7248.000000,7248.000000,7248.000000,7248.000000
mean,12.902528,0.060559,140.599344,1998.729443,2441.282926,17.311534
std,4.628120,0.044552,62.075950,8.233890,1660.758040,8.272396
min,4.555000,0.000000,31.290000,1985.000000,69.243200,2.000000
25%,8.895000,0.026014,93.743600,1987.000000,1188.453000,11.000000
50%,12.650000,0.049824,141.998300,1999.000000,2063.980000,16.000000
75%,16.850000,0.088182,185.635150,2004.000000,3307.694400,22.000000
max,21.350000,0.188620,266.888400,2009.000000,13086.964800,57.000000


In [400]:
df_h_no_moda.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,productos
6863,DRA24,19.350,Regular,0.039990,Soft Drinks,165.0868,OUT049,1999,Tier 1,Supermarket Type1,982.7208,6.0
3927,DRA59,8.270,Regular,0.128127,Soft Drinks,183.6924,OUT049,1999,Tier 1,Supermarket Type1,1295.6468,7.0
3465,DRB01,7.390,Low Fat,0.082367,Soft Drinks,187.7530,OUT049,1999,Tier 1,Supermarket Type1,1518.0240,8.0
6626,DRB13,6.115,Regular,0.007055,Soft Drinks,188.6530,OUT049,1999,Tier 1,Supermarket Type1,3605.3070,19.0
2657,DRB24,8.785,Low Fat,0.020609,Soft Drinks,155.1656,OUT049,1999,Tier 1,Supermarket Type1,4016.1056,26.0


In [401]:
df_h_no_moda_3qt=df_h_no_moda[df_h_no_moda["Item_Outlet_Sales"]>3307.694400]
df_h_no_moda_3qt.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,productos
6626,DRB13,6.115,Regular,0.007055,Soft Drinks,188.6530,OUT049,1999,Tier 1,Supermarket Type1,3605.3070,19.0
2657,DRB24,8.785,Low Fat,0.020609,Soft Drinks,155.1656,OUT049,1999,Tier 1,Supermarket Type1,4016.1056,26.0
4730,DRC27,13.800,Low Fat,0.058193,Dairy,246.9802,OUT049,1999,Tier 1,Supermarket Type1,5896.3248,24.0
7697,DRD60,15.700,Low Fat,0.037290,Soft Drinks,182.7634,OUT049,1999,Tier 1,Supermarket Type1,3453.5046,19.0
915,DRE60,9.395,Low Fat,0.159582,Soft Drinks,224.7720,OUT049,1999,Tier 1,Supermarket Type1,7017.5320,31.0


In [402]:
dff=df_h_no_moda_3qt.groupby(["Item_Type","Item_Identifier"])["Item_Outlet_Sales"].agg(["count","mean","sum","median"]).reset_index()
dff

,Item_Type,Item_Identifier,count,mean,sum,median
0,Baking Goods,FDA35,2,3920.23040,7840.4608,3920.2304
1,Baking Goods,FDA47,2,4812.06950,9624.1390,4812.0695
2,Baking Goods,FDA48,2,4323.37230,8646.7446,4323.3723
3,Baking Goods,FDB36,2,3541.39020,7082.7804,3541.3902
4,Baking Goods,FDB37,5,5143.57132,25717.8566,4566.7222
...,...,...,...,...,...,...
826,Starchy Foods,FDF47,2,3477.80630,6955.6126,3477.8063
827,Starchy Foods,FDG47,3,5508.82920,16526.4876,4984.1788
828,Starchy Foods,FDH35,2,4933.57800,9867.1560,4933.5780
829,Starchy Foods,FDI35,2,4998.49350,9996.9870,4998.4935


In [403]:
import plotly.express as px


da = px.data.gapminder()
fig = px.scatter(dff, x="sum", y="Item_Type", size="count", color="Item_Type",
           hover_name="Item_Identifier", size_max=15)

fig.show()

In [404]:
df.head(3)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,productos
6863,DRA24,19.35,Regular,0.039990,Soft Drinks,165.0868,OUT049,1999,Tier 1,Supermarket Type1,982.7208,6.0
3927,DRA59,8.27,Regular,0.128127,Soft Drinks,183.6924,OUT049,1999,Tier 1,Supermarket Type1,1295.6468,7.0
3465,DRB01,7.39,Low Fat,0.082367,Soft Drinks,187.7530,OUT049,1999,Tier 1,Supermarket Type1,1518.0240,8.0



# 3. Prepocesamiento de los datos

In [405]:
df.head(3)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,productos
6863,DRA24,19.35,Regular,0.039990,Soft Drinks,165.0868,OUT049,1999,Tier 1,Supermarket Type1,982.7208,6.0
3927,DRA59,8.27,Regular,0.128127,Soft Drinks,183.6924,OUT049,1999,Tier 1,Supermarket Type1,1295.6468,7.0
3465,DRB01,7.39,Low Fat,0.082367,Soft Drinks,187.7530,OUT049,1999,Tier 1,Supermarket Type1,1518.0240,8.0


In [406]:
#df.Outlet_Type.unique()

In [407]:
#df  = df.loc[~(df['Outlet_Type'] == "Grocery Store"), :]

In [408]:
#df.Outlet_Type.unique()

In [409]:
X = df.drop(["Item_Outlet_Sales","Item_Identifier","Outlet_Establishment_Year"],axis=1)
y = df[["Item_Outlet_Sales"]]

X["Outlet_Type"]  = X["Outlet_Type"].map({'Supermarket Type1':1, 'Supermarket Type2':1, 'Supermarket Type3':1,'Grocery Store':0})


In [410]:
X.head(3)

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Location_Type,Outlet_Type,productos
6863,19.35,Regular,0.039990,Soft Drinks,165.0868,OUT049,Tier 1,1,6.0
3927,8.27,Regular,0.128127,Soft Drinks,183.6924,OUT049,Tier 1,1,7.0
3465,7.39,Low Fat,0.082367,Soft Drinks,187.7530,OUT049,Tier 1,1,8.0


In [411]:
from sklearn.model_selection import train_test_split

np.random.seed(42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [412]:
Xtr_num = X_train.select_dtypes(include=["int64", "float64"])
Xte_num = X_test.select_dtypes(include=["int64", "float64"])

Xtr_num_1 = Xtr_num[["Outlet_Type"]]
Xte_num_1 = Xte_num[["Outlet_Type"]]

Xtr_num   = Xtr_num.drop("Outlet_Type", axis=1)
Xte_num   = Xte_num.drop("Outlet_Type", axis=1)

In [413]:
from sklearn.preprocessing import StandardScaler

sc  = StandardScaler()
Xtr_num_n         = sc.fit_transform(Xtr_num)
Xtr_num_n         = pd.DataFrame(Xtr_num_n)
Xtr_num_n.columns = Xtr_num.columns

Xte_num_n         = sc.transform(Xte_num)
Xte_num_n         = pd.DataFrame(Xte_num_n)
Xte_num_n.columns = Xte_num.columns

In [414]:

Xtr_cat = X_train.select_dtypes(exclude=["int64", "float64"])
Xte_cat = X_test.select_dtypes(exclude=["int64", "float64"])


In [415]:
def dummies_cat(X_tt):
  X_cat = pd.DataFrame()

  for nn in list(X_tt.columns):
    dummies = pd.get_dummies(X_tt[nn],  dtype=int)
    X_cat=  pd.concat([X_cat,dummies],axis=1)
  return X_cat

In [416]:
Xtr_cat_n   = dummies_cat(Xtr_cat)
Xte_cat_n   = dummies_cat(Xte_cat)

In [417]:
Xtr_num_n.index = Xtr_cat_n.index
Xte_num_n.index = Xte_cat_n.index

In [418]:
X_train     = pd.concat([Xtr_num_n,Xtr_cat_n,Xtr_num_1], axis=1)
X_test      = pd.concat([Xte_num_n,Xte_cat_n,Xte_num_1], axis=1)

# 4. Modelos

## 4.1 Primer grupo de modelos

### 4.1.1. Entrenamiento con KNN

In [419]:
y_test

,Item_Outlet_Sales
460,284.2966
6713,2266.3832
7265,1480.0734
2793,1837.6080
5718,2664.5316
...,...
7357,1549.9824
7879,3962.8416
6395,2871.5954
5211,2153.1972


In [420]:
from sklearn.neighbors import KNeighborsRegressor

In [421]:
param_grid = [{"n_neighbors":np.arange(1,30)}
]

knn_reg = KNeighborsRegressor()

grid_search = GridSearchCV(knn_reg, param_grid, cv=5,
                           return_train_score=True)

grid_search.fit(X_train, y_train)


GridSearchCV(cv=5, estimator=KNeighborsRegressor(),
             param_grid=[{'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])}],
             return_train_score=True)

In [422]:
cvres = grid_search.cv_results_
for values in zip(cvres['mean_test_score'], cvres['params']):
    print(values)

(0.7739310961215766, {'n_neighbors': 1})
(0.8427876587277812, {'n_neighbors': 2})
(0.8704077798364942, {'n_neighbors': 3})
(0.8847054197884139, {'n_neighbors': 4})
(0.8903776353890154, {'n_neighbors': 5})
(0.8951703948705363, {'n_neighbors': 6})
(0.899309559931029, {'n_neighbors': 7})
(0.9018424831824777, {'n_neighbors': 8})
(0.9026220080718981, {'n_neighbors': 9})
(0.904009165772505, {'n_neighbors': 10})
(0.905288795951266, {'n_neighbors': 11})
(0.9053436310395752, {'n_neighbors': 12})
(0.9053061214831228, {'n_neighbors': 13})
(0.9054549507472739, {'n_neighbors': 14})
(0.9046009288858169, {'n_neighbors': 15})
(0.9041385366961185, {'n_neighbors': 16})
(0.9037277340813861, {'n_neighbors': 17})
(0.9032170825364876, {'n_neighbors': 18})
(0.9023034057632682, {'n_neighbors': 19})
(0.9016502998816426, {'n_neighbors': 20})
(0.9011695958175068, {'n_neighbors': 21})
(0.9000912794000451, {'n_neighbors': 22})
(0.8993755159710725, {'n_neighbors': 23})
(0.8985182690442572, {'n_neighbors': 24})
(0.8

Mejor resultado (0.9054549507472739, {'n_neighbors': 14})

In [423]:
knn_reg = KNeighborsRegressor(n_neighbors= 14)
knn_reg.fit(X_train, y_train)
y_pred=knn_reg.predict(X_test)

print(f"score de prueba: {knn_reg.score(X_train, y_train)}")
print(f"score de test: {knn_reg.score(X_test, y_test)}")

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("===========")
print("RMSE:", rmse)
print("MSE:", mse)
print("MAE:", mae)


score de prueba: 0.9224407067107521
score de test: 0.9087200883651372
RMSE: 506.284702656946
MSE: 256324.20014443222
MAE: 365.81050999497177


### 4.1.2. Modelo de Arbol

In [428]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

param_grid = [{"max_depth":np.arange(2,37)}
]

dtc_reg = DecisionTreeRegressor()

grid_search = GridSearchCV(dtc_reg, param_grid, cv=5,
                           return_train_score=True)

grid_search.fit(X_train, y_train)



GridSearchCV(cv=5, estimator=DecisionTreeRegressor(),
             param_grid=[{'max_depth': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36])}],
             return_train_score=True)

In [429]:
cvres = grid_search.cv_results_
for values in zip(cvres['mean_test_score'], cvres['params']):
    print(values)

(0.6709981191494647, {'max_depth': 2})
(0.8317031852033034, {'max_depth': 3})
(0.9046054177256997, {'max_depth': 4})
(0.9535735796449609, {'max_depth': 5})
(0.9728601697922485, {'max_depth': 6})
(0.9847141403684929, {'max_depth': 7})
(0.9906255442332675, {'max_depth': 8})
(0.993516103671715, {'max_depth': 9})
(0.9937554134932919, {'max_depth': 10})
(0.9947036561389575, {'max_depth': 11})
(0.9945079205060907, {'max_depth': 12})
(0.9947562587144321, {'max_depth': 13})
(0.9943464800969732, {'max_depth': 14})
(0.9948054788339118, {'max_depth': 15})
(0.9947735635059006, {'max_depth': 16})
(0.9946659283633664, {'max_depth': 17})
(0.9948418744805411, {'max_depth': 18})
(0.9947896072429101, {'max_depth': 19})
(0.994759845986897, {'max_depth': 20})
(0.9948374410766145, {'max_depth': 21})
(0.994550433235086, {'max_depth': 22})
(0.9950890098923398, {'max_depth': 23})
(0.9948276913065863, {'max_depth': 24})
(0.9947027613505913, {'max_depth': 25})
(0.9946191093882746, {'max_depth': 26})
(0.99468012

Resultado:  (0.9950890098923398, {'max_depth': 23})

In [430]:
dtc_reg = DecisionTreeRegressor(max_depth=23)
dtc_reg.fit(X_train, y_train)
y_pred=knn_reg.predict(X_test)

print(f"score de prueba: {dtc_reg.score(X_train, y_train)}")
print(f"score de test: {dtc_reg.score(X_test, y_test)}")

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("===========")
print("RMSE:", rmse)
print("MSE:", mse)
print("MAE:", mae)


score de prueba: 1.0
score de test: 0.9965413990353026
RMSE: 506.284702656946
MSE: 256324.20014443222
MAE: 365.81050999497177


### 4.1.3. RandomForest

In [431]:
param_grid = [{'max_depth': [8,12,16,20,24,30],
              'n_estimators': [10,50,200]}
]

rf_reg = RandomForestRegressor()

grid_search = GridSearchCV(rf_reg, param_grid, cv=5,
                           return_train_score=True)

grid_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/usr/local/lib/python3.10/dist-packa

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid=[{'max_depth': [8, 12, 16, 20, 24, 30],
                          'n_estimators': [10, 50, 200]}],
             return_train_score=True)

In [338]:
cvres = grid_search.cv_results_
for values in zip(cvres['mean_test_score'], cvres['params']):
    print(values)

(0.7296281892120933, {'max_depth': 2, 'n_estimators': 10})
(0.7329809495453427, {'max_depth': 2, 'n_estimators': 50})
(0.7310418824610229, {'max_depth': 2, 'n_estimators': 200})
(0.9494196413141216, {'max_depth': 4, 'n_estimators': 10})
(0.9508755317053023, {'max_depth': 4, 'n_estimators': 50})
(0.952400252780316, {'max_depth': 4, 'n_estimators': 200})
(0.9898525318228814, {'max_depth': 6, 'n_estimators': 10})
(0.9915506040583992, {'max_depth': 6, 'n_estimators': 50})
(0.9919440900640248, {'max_depth': 6, 'n_estimators': 200})
(0.9969129798503907, {'max_depth': 8, 'n_estimators': 10})
(0.9972548913148944, {'max_depth': 8, 'n_estimators': 50})
(0.9974077608243214, {'max_depth': 8, 'n_estimators': 200})
(0.9975938622820486, {'max_depth': 10, 'n_estimators': 10})
(0.9979674086465886, {'max_depth': 10, 'n_estimators': 50})
(0.9980297923265493, {'max_depth': 10, 'n_estimators': 200})
(0.9977170823670228, {'max_depth': 13, 'n_estimators': 10})
(0.9979651916439713, {'max_depth': 13, 'n_estima

Resultado: (0.9980188884512726, {'max_depth': 13, 'n_estimators': 200})

In [432]:
rf_reg = RandomForestRegressor(max_depth=13,n_estimators=200)
rf_reg.fit(X_train, y_train)
y_pred=rf_reg.predict(X_test)

print(f"score de prueba: {rf_reg.score(X_train, y_train)}")
print(f"score de test: {rf_reg.score(X_test, y_test)}")

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("===========")
print("RMSE:", rmse)
print("MSE:", mse)
print("MAE:", mae)


<ipython-input-432-1613a5ad9f7f>:2: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



score de prueba: 0.9997299100717156
score de test: 0.9983072686602573
RMSE: 68.94475258005781
MSE: 4753.378908325389
MAE: 26.245874802276433
